In [1]:
import random
import numpy as np
from collections import deque
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import math

import sys
sys.path.append('../')

from util import *


In [2]:
max_reward = 100
step_penalty = -1

def getReward(matrix):
    if (matrix==np.identity(2)).all():
        return max_reward
    else:
        return step_penalty

In [3]:
df = pd.read_csv("../Data_Generation/Data_files/sl2_Z.csv")

In [4]:
df[(df['val1'] % 2 == 1) & (df['val2'] % 2 == 0) & (df['val3'] % 2 == 0) & (df['val4'] % 2 == 1)]

In [5]:
filter_df = df[df['val1'] % 2 == 1]
filter_df = filter_df[filter_df['val2'] % 2 == 0]
filter_df = filter_df[filter_df['val3'] % 2 == 0]
filter_df = filter_df[filter_df['val4'] % 2 == 1]
filter_df

In [6]:
env = TabularQEnv(k_sl2z_2s_gen, defaultdict(lambda: 0), getReward, max_reward)

EPISODES = 3000
LEARNING_RATE = .9
DISCOUNT_FACTOR = .99
EPSILON = 1
EPSILON_DECAY = .9999

random.seed(42)

# starts with an estimate of zero reward for each state.
# adapted from ChatGPT
episode_reward_record = deque(maxlen=500)

for i in range(EPISODES):
    episode_reward = 0
    done = False
    # choose a random starting row
    # adapted from https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe
    cur_row = df.sample(1)
    obs = np.array([
        [int(cur_row['val1']), int(cur_row['val2'])], 
        [int(cur_row['val3']), int(cur_row['val4'])]
        ])

    index = 1

    while (not done):
        # perform an epsilon greedy action 
        # Q(s, a) = (1-LEARNING_RATE)Q(s, a) + (LEARNING_RATE)(r + DISCOUNT_FACTOR(max a'(Q(s', a'))))
        obs, reward, done = env.step(LEARNING_RATE, DISCOUNT_FACTOR, EPSILON, obs)

        episode_reward += reward # update episode reward

        index += 1
        # if we take more than 100 steps, end this iteration early (we are probably not making progress)
        if index > 100:
            done=True

    # decay the epsilon
    EPSILON *= EPSILON_DECAY

    # record the reward for this episode
    episode_reward_record.append(episode_reward) 

    if (i+1)%500 ==0 and i>0:
        print("Episode {i}\teps: {eps:.3f}\tAvg rwd: {rwd:.3f}".format(i=i+1, eps=EPSILON, rwd=sum(list(episode_reward_record))/500))



In [7]:
def access_Q_table(mat):
    return env.Q_table[matrix_to_tuple(mat)]

In [8]:
access_Q_table(np.array([[1, 1], [0, 1]]))

In [9]:
# test with the other dataframe. 
test_df = pd.read_csv("../Data_Generation/Data_files/sl2_Z_test.csv")
test_df['num_moves_Q_learning_needs'] = test_df.apply(lambda row: env.play(df_row_to_mat(row)), axis=1)

In [10]:
test_df['num_moves_Q_learning_needs'] = test_df.apply(lambda row: env.play(df_row_to_mat(row)), axis=1)

In [12]:
print("The proportion of starting positions in the test dataset that we can find a route to the origin that's <50 steps: ")
sum(test_df['num_moves_Q_learning_needs']!=100)/test_df.shape[0]

print("Of these, the proportion of times where we learned a path that was < 20 moves: ")
# encouraging because all of these were generated as sequences of 30 moves
# so we've found significantly faster paths back to the origin for almost all moves that we find a path to the origin 
sum(test_df['num_moves_Q_learning_needs']<20)/sum(test_df['num_moves_Q_learning_needs']!=100)

Of these, the proportion of times where we learned a path that was < 20 moves: 


1.0

In [15]:
append_info_states_csv("../Data_Generation/Data_files/sl2_Z_test.csv", 
                       "../Data_Generation/Data_files/train_rows_SL2Z_Q_learn.csv", 
                       "../Data_Generation/Data_files/test_rows_SL2Z_Q_learn.csv")